# 2019 1st ML month with KaKR
## 캐글 코리아와 함께하는 1st ML 대회 - 타이타닉 생존자를 예측하라!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
sns.set(style='white', context='notebook', palette='deep')
%matplotlib inline

In [2]:
# Load train and Test set
train = pd.read_csv("./kaggle_titanic/train.csv")
test = pd.read_csv("./kaggle_titanic/test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data preprocessing

In [5]:
# Check Nan values of each columns
print(train.isna().sum())
print("-----------------")
print(test.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
-----------------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [6]:
# Drop 'Name' column since it doesn't have any correlation
train.drop(['Name'], axis=1, inplace=True)
test.drop(['Name'], axis=1, inplace=True)

In [7]:
# Drop 'Cabin' column, since it is a categorical value but there are too many Nans
train.drop(['Cabin'], axis=1, inplace=True)
test.drop(['Cabin'], axis=1, inplace=True)

In [8]:
# Fill Nan values of 'Age' column by its mean value
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)

In [9]:
# Drop rows which has Nan value for 'Embarked' column
train.dropna(subset=['Embarked'], inplace=True)
train.shape

(889, 10)

In [10]:
# Fill Nan value of 'Fare' column in test set by its mean value
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [11]:
# Drop 'Ticket' column
train.drop(['Ticket'], axis=1, inplace=True)
test.drop(['Ticket'], axis=1, inplace=True)

### Convert categorical data to numeric data

In [12]:
# Sex
def convertSex(sex):
    return 1 if sex == 'male' else 0

train['Sex'] = train['Sex'].apply(convertSex)
test['Sex'] = test['Sex'].apply(convertSex)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,0,38.0,1,0,71.2833,C
2,3,1,3,0,26.0,0,0,7.9250,S
3,4,1,1,0,35.0,1,0,53.1000,S
4,5,0,3,1,35.0,0,0,8.0500,S


In [13]:
embarked_group = train.groupby('Embarked')
print(embarked_group.size())

Embarked
C    168
Q     77
S    644
dtype: int64


In [14]:
# Embarked
def convertEmbarked(embarked):
    if embarked == 'C':
        return 0
    elif embarked == 'Q':
        return 1
    else:
        return 2

train['Embarked'] = train['Embarked'].apply(convertEmbarked)
test['Embarked'] = test['Embarked'].apply(convertEmbarked)

train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [22]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1
1,893,3,0,47.0,1,0,7.0000,2
2,894,2,1,62.0,0,0,9.6875,1
3,895,3,1,27.0,0,0,8.6625,2
4,896,3,0,22.0,1,1,12.2875,2


## Train

In [25]:
X_train, X_valid = train[:int(train.shape[0] * 0.7)], train[int(train.shape[0] * 0.7):]
train_x = X_train.drop(['Survived'], axis=1)
train_y = X_train.loc[:,['Survived']]
valid_x = X_valid.drop(['Survived'], axis=1)
valid_y = X_valid.loc[:,['Survived']]

In [35]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

ModuleNotFoundError: No module named 'keras'

In [31]:
output_dim = nb_classes = 2
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 
batch_size = 128 
nb_epoch = 20


EnvironmentLocationNotFound: Not a conda environment: /Users/user/miniconda3/envs/edu

